<a href="https://colab.research.google.com/github/ma-serra/AGPrec_2024/blob/main/Dados_externos_arquivos_locais%2C_Drive%2C_Planilhas_e_Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook contém instruções para carregar e salvar dados de fontes externas.

# Sistema de arquivos local

## Fazer upload de arquivos a partir de sistema de arquivos local

<code>files.upload</code> retorna um dicionário com os arquivos incluídos no upload.
O dicionário é indexado com o nome do arquivo e os valores são os dados enviados por upload.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Baixar arquivos para o sistema de arquivos local

<code>files.download</code> invocará o download do arquivo pelo navegador para seu computador local.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

É possível acessar os arquivos no Drive de várias maneiras. Por exemplo:
- Montar o Google Drive na máquina virtual do ambiente de execução
- Usar um wrapper ao redor da API, como o <a href="https://docs.iterative.ai/PyDrive2/">PyDrive2</a>
- Usar a <a href="https://developers.google.com/drive/v3/web/about-sdk">API REST nativa</a>



Veja abaixo exemplos de cada um eles.

## Montar o Google Drive localmente

O exemplo abaixo mostra como montar o Google Drive no seu ambiente de execução usando um código de autorização, além de como gravar e ler arquivos nele. Depois de executado, você verá o novo arquivo &#40;<code>foo.txt</code>&#41; no <a href="https://drive.google.com/">https://drive.google.com/</a>.

Isto permite somente ler, gravar e mover arquivos. Para modificar de maneira programática as configurações de compartilhamento ou outros metadados, use uma das opções abaixo.

<strong>Observação:</strong> ao usar o botão "Montar Drive" no navegador de arquivos, não é necessário usar nenhum código de autenticação para notebooks que tenham sido editados somente pelo usuário atual.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive2

Os exemplos abaixo demonstram a autenticação e o upload/download de arquivos usando o PyDrive2. Mais exemplos estão disponíveis na <a href="https://docs.iterative.ai/PyDrive2/">documentação do PyDrive2</a>.

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autentique e crie o cliente do PyDrive2.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Crie e faça upload de um arquivo de texto.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


Use o ID do arquivo para carregá-lo e imprima seu conteúdo.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## API REST do Drive

Para usar a API REST do Drive, primeiro é necessário autenticar e construir um cliente da API.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

Com esse cliente, é possível usar todas as funções da <a href="https://developers.google.com/drive/v3/reference/">Referência da API do Google Drive</a>. Veja os exemplos a seguir.


### Criar um novo arquivo do Drive com dados do Python

Primeiro, crie um arquivo local para fazer upload.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Faça upload usando o método <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a>. Mais detalhes sobre como fazer upload de arquivos estão disponíveis na <a href="https://developers.google.com/drive/v3/web/manage-uploads">documentação do desenvolvedor</a>.

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


Depois de executar a célula acima, você verá um novo arquivo chamado "Sample file" no <a href="https://drive.google.com/">https://drive.google.com/</a>.

### Baixar dados de um arquivo do Drive para Python

Faça do download do arquivo enviado por upload anteriormente.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


Para baixar um arquivo diferente, configure o <code>file&#95;id</code> acima com o ID desse arquivo, que será similar a este: "1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz".

# Planilhas Google


## Extensão do Workspace para as Planilhas Google

Temos uma extensão do Workspace, <a href="https://workspace.google.com/u/0/marketplace/app/sheets_to_colab/945625412720">Planilhas para Colab</a>, que permite importar dados diretamente das Planilhas Google para o Colab usando a interface das Planilhas. Para saber mais, acesse o link da extensão.

## Como interagir com as Planilhas Google usando o gspread

Você também pode usar a biblioteca <a href="https://github.com/burnash/gspread"><code>gspread</code></a> de código aberto para interagir com as Planilhas Google. O código abaixo mostra como configurar e autenticar o <code>gspread</code>.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Veja abaixo alguns exemplos de <code>gspread</code>. Há outros exemplos disponíveis na <a href="https://github.com/burnash/gspread#more-examples">página sobre <code>gspread</code> do GitHub</a>.

### Criar uma nova planilha com dados do Python

In [ ]:
sh = gc.create('My cool spreadsheet')

Depois de executar a célula acima, você verá uma nova planilha chamada "My cool spreadsheet" no <a href="https://sheets.google.com/">https://drive.google.com/</a>.

Abra sua nova planilha e adicione alguns dados aleatórios.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

### Fazer o download de dados de uma planilha em Python como um DataFrame da pandas

Leia novamente os dados aleatórios inseridos anteriormente e converta o resultado para um <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">Pandas DataFrame</a>.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# InteractiveSheet

Agora é possível incorporar as Planilhas Google em tempo real no Colab com a biblioteca <code>InteractiveSheet</code>. Isso significa que você pode criar e editar dados nas Planilhas Google, além de fazer a incorporação ao seu notebook com DataFrames do Pandas. Tudo isso pelo Colab.

In [ ]:
from google.colab import sheets

# Create a new interactive sheet and add data to it.
sheet = sheets.InteractiveSheet()

In [ ]:
# Get a Pandas DataFrame from the selected worksheet
df = sheet.as_df()

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))

# Create a new sheet and include the column names as the first row.
sheet = sheets.InteractiveSheet(df=df, title='foo', include_column_headers=True)

In [ ]:
# Push data from Colab to the selected worksheet
df2 = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
sheet.update(df=df)

In [ ]:
# Display the sheet in the output of the current cell
sheet.display()

# Google Cloud Storage &#40;GCS&#41;

Para usar o Colaboratory com GCS, será preciso criar um <a href="https://cloud.google.com/storage/docs/projects">projeto do Google Cloud</a> ou usar um preexistente.

Especifique abaixo o ID do projeto:

In [ ]:
project_id = 'Your_project_ID_here'

Os arquivos no GCS são contidos em <a href="https://cloud.google.com/storage/docs/buckets">buckets</a>.

Os buckets precisam ter nomes exclusivos mundialmente. Por isso, vamos gerar um aqui.

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

Para acessar o GCS, é preciso autenticar.

In [ ]:
from google.colab import auth
auth.authenticate_user()

O GCS pode ser acessado pelo utilitário <code>gsutil</code> da linha de comando ou através da API Python nativa.

## `gsutil`

Primeiro, é preciso configurar <code>gsutil</code> para usar o projeto especificado anteriormente usando <code>gcloud</code>.

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


Crie um arquivo local para fazer upload.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Faça um bucket para onde você quer fazer o upload &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">acesse a documentação aqui</a>&#41;.

In [ ]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


Copie o arquivo para o novo bucket &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">acesse a documentação aqui</a>&#41;.

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


Despeje o conteúdo do arquivo recém copiado para conferir se tudo funcionou corretamente &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">acesse a documentação aqui</a>&#41;.


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [ ]:
# @markdown Quando o upload for concluído, os dados aparecerão no navegador do armazenamento do Cloud Console do seu projeto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Por último, faça download do arquivo que acaba de ser enviado no exemplo acima. É muito simples: basta inverter a ordem do comando <code>gsutil cp</code>.

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt

# Imprima o resultado para conferir se a transferência funcionou.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## API do Python

Estes snippets se baseiam em <a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">um exemplo maior</a> que mostra outros usos da API.

Primeiro, crie o cliente do serviço.

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Crie um arquivo local para fazer upload.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Crie um bucket no projeto especificado acima.

In [ ]:
# Use um nome de bucket exclusivo mundialmente do exemplo gsutil acima.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


Faça upload do arquivo para o bucket recém criado.

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name,
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [ ]:
# @markdown Quando o upload for concluído, os dados aparecerão no navegador do armazenamento do Cloud Console do seu projeto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Baixe o arquivo que acaba de ser enviado.

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


Inspecione o arquivo salvo.


In [ ]:
!cat /tmp/downloaded_from_gcs.txt

my sample file